<a href="https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/2_zaklady-pythonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Úvod a cíle kapitoly

V této kapitole se podíváme na získání dat přes tzv. API, neboli "application programming interface". Webový prohlížeč je jen jeden ze způsobů, jak využívat internet. Mnohé aplikace, které máme v našich počítačích i chtrých telefonech, získávají z internetu data, aniž by webový prohlížeč jakkoli vstupoval do hry. Tyto aplikace komunikují s WWW servery za využití tzv. HTTP (Hypertext Transfer Protocol): Aplikace vznášejí dotazy v podobě URL adres a server na ně odpovídá tím, že vrátí např. dotazovaná data. Tento systém dotazování v podobě URL adres a odpovídání na ně v podobě dat je nikdy standardizovaný do podoby standardizovaného API. Tato technologie je využívána i pro potřeby zpřístupňování dat z oblasti péče o kulturní dědictví. 

Vezměme příklad: Projekt Epigraphic Database Heidelberg hostí digitalizovanou kolekci více než 80,000 převážně latinských nápisů z antického Říma. Tyto nápisy si veřejnost může prohlížet a prohledávat za využití webových stránek [zde](https://edh.ub.uni-heidelberg.de/inschrift/suche). Kromě toho má však badatel ještě jinou možnost, jak se dostat k příslušným datům, a totiž právě za využití speciálně vyvinutého API, které je zdokumentováno [zde](https://edh.ub.uni-heidelberg.de/data/api).

V této dokumentaci se podrobně dočteme, jaké parametry můžeme v našem dotazu (query) použít a jak. 

Základní URL adresa je `https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?`, za níž připojujeme parametry pro určení námi hledané skupiny nápisů.

Chceme-li např. získat data o všech nápisech z Římské provincie *Germania superior*, nastavíme parametr "provinz" na hodnotu "ges". Výsledná URL adresa pak vypadá takto:

```
https://edh.ub.uni-heidelberg.de/data/api/inschrift/suche?provinz=ges
```